In [ ]:
import os
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.sequence import pad_sequences # to pad sequences

from sklearn.preprocessing import StandardScaler # to normalise data
from sklearn.metrics import mean_squared_error

2023-05-01 12:36:51.357062: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-01 12:36:52.180701: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
#========================#
# PACKAGE VERSIONS       #
#========================#

# print(python_version())
# 3.9.12

# !pip show tensorflow
# Name: tensorflow
# Version: 2.12.0

# print('\n'.join(f'{m.__name__}=={m.__version__}' for m in globals().values() if getattr(m, '__version__', None)))
# pandas==2.0.0
# numpy==1.23.5
# tensorflow==2.12.0
# keras.api._v2.keras==2.12.0

In [ ]:
# check working directory (should be .../EPiC-challenge)
cwd = os.getcwd()
print(cwd)

# change working directory if needed
# DIR = "set working directory here"
# os.chdir(DIR)

/home/jc2822/EPiC-challenge


In [ ]:
#========================#
# LOAD DATA              #
#========================#
#
# Variable Names:
# train_data
# test_data
#

# load each fold as dictionary
def load_data_with_folds(scenario_dir_path, dataset_type):
    # make dict to store data frames for each fold
    fold_dfs = {}
    # iterate over the scenario directory
    for fold_dir in Path(scenario_dir_path).iterdir():
        if fold_dir.name.startswith('.'):
            continue  # skip hidden files/folders like .DS_Store and .ipynb_checkpoints
        train_annotations_dir = Path(fold_dir, f"{dataset_type}/annotations/")
        train_physiology_dir = Path(fold_dir, f"{dataset_type}/physiology/")
        train_physiology_files = sorted(p for p in Path(train_physiology_dir).iterdir() if not p.name.startswith('.'))
        train_annotation_files = sorted(p for p in Path(train_annotations_dir).iterdir() if not p.name.startswith('.'))
        fold_df = pd.DataFrame()
        for physiology_file_path, annotations_file_path in zip(train_physiology_files, train_annotation_files):
            if physiology_file_path.name.startswith('.') or annotations_file_path.name.startswith('.'):
                continue  # skip hidden files like .DS_Store and .ipynb_checkpoints
            assert physiology_file_path.name == annotations_file_path.name, "Order mismatch"
            df_physiology = pd.read_csv(physiology_file_path)
            df_annotations = pd.read_csv(annotations_file_path)
            subject = annotations_file_path.name
            merged_df = pd.merge(df_physiology, df_annotations, on="time", how="outer")
            merged_df["subject"] = [subject] * len(merged_df)
            fold_df = pd.concat([fold_df, merged_df], ignore_index=True)
        fold_dfs[fold_dir.name] = fold_df
    return fold_dfs

# specify scenario path
scenario_dir = "data/scenario_2"

# train data
print("Loading train data")
train_data = load_data_with_folds(scenario_dir, "train")

# test data
print("Loading test data")
test_data = load_data_with_folds(scenario_dir, "test")


Loading train data
Loading test data


In [ ]:
#====================================#
# PAD VIDEO SEQUENCES: TRAINING SET  #
#====================================#
#
# Variable Names:
# train_data_scaled_padded
# test_data_scaled_padded
#

# set maxlen so that it is in increments of 50 to match the affect timescales
maxlen = 130000 

# separate out each video sequence, in prep for padding
train_data_scaled_padded = {}

# loop through each fold
for key, df in train_data.items():
    
    # find start of each sequence
    idx_zero = np.where(df['time']==0)[0]

    # initialize lists to store the data
    valence = []
    arousal = []
    raw_data = []

    # loop through each video sequence in the fold
    for i in range(len(idx_zero)):
        
        if i == len(idx_zero) - 1:
            
            valence.append(df['valence'][idx_zero[i]:])
            arousal.append(df['arousal'][idx_zero[i]:])
            raw_data.append(df.iloc[idx_zero[i]:, 1:9])
            
        else:
            
            valence.append(df['valence'][idx_zero[i]:idx_zero[i+1]])
            arousal.append(df['arousal'][idx_zero[i]:idx_zero[i+1]])
            raw_data.append(df.iloc[idx_zero[i]:idx_zero[i+1], 1:9])

    # store in each fold in dictionary
    train_data_scaled_padded[key] = {
        'valence': np.array(valence, dtype=object),
        'arousal': np.array(arousal, dtype=object),
        'raw_data': np.array(raw_data, dtype=object)
    }

# pad train data sequences to make them equal length
for key, array in train_data_scaled_padded.items():
    
    array['valence'] = pad_sequences(valence, dtype="float64", maxlen=maxlen, value=0.0, padding="post")
    array['arousal'] = pad_sequences(arousal, dtype="float64", maxlen=maxlen, value=0.0, padding="post")
    array['raw_data'] = pad_sequences(raw_data, dtype="float64", maxlen=maxlen, value=0.0, padding="post")

# check shape after padding
for key, array in train_data_scaled_padded.items():  
    
    print(array['valence'].shape)
    print(array['arousal'].shape)
    print(array['raw_data'].shape)
    break
    
for seq in array['valence']:
    length = len(seq)
    print(f"Length of sequence: {length}")
    break

(192, 130000)
(192, 130000)
(192, 130000, 8)
Length of sequence: 130000


In [ ]:
# reshape data to prepare for timeseries batching
for key, array in train_data_scaled_padded.items():
    array['valence'] = array['valence'].reshape(-1)
    array['arousal'] = array['arousal'].reshape(-1)
    array['raw_data'] = array['raw_data'].reshape(-1, 8)
    
# check shape again
for key, array in train_data_scaled_padded.items():  
    print(array['valence'].shape)
    print(array['arousal'].shape)
    print(array['raw_data'].shape)
    break


(24960000,)
(24960000,)
(24960000, 8)


In [ ]:
# stack valence and arousal & transpose
for key, array in train_data_scaled_padded.items(): 
    array['affect'] = np.vstack((array['valence'], array['arousal'])).T

In [ ]:
#================================#
# PAD VIDEO SEQUENCES: TEST SET  #
#================================#


# pad test data
test_data_scaled_padded = {}

# loop through each fold
for key, df in test_data.items():
    
    # find start of each video sequence
    idx_start = np.where(df['time']==9900)[0] # only keep buffer of 100ms prior
    idx_end = np.where(df['time']==40000)[0]

    # initialize lists to store the data
    raw_data = []

    # loop through each sequence in fold
    for i in range(len(idx_start)): # n_videos
        
        raw_data.append(df.iloc[idx_start[i]:idx_end[i], 1:9])

    # store in each fold in dictionary
    test_data_scaled_padded[key] = {
        'raw_data': np.array(raw_data, dtype=object)
    }

# pad test data sequences
for key, array in test_data_scaled_padded.items():
    array['raw_data'] = pad_sequences(raw_data, dtype="float64", maxlen=30100, value=0.0, padding="post")
    array['raw_data'] = array['raw_data'].reshape(-1, 8)

# check shape after padding
for key, array in test_data_scaled_padded.items():  
    
    print(array['raw_data'].shape)
    break
    
for seq in array['raw_data']:
    length = len(seq)
    print(f"Length of sequence: {length}")
    break

(1444800, 8)
Length of sequence: 8


In [ ]:
#================================#
# SPLIT INTO TIMESERIES BATCHES  #
#================================#
#
# Variable Names:
# train_dataset
# test_dataset
#

# split into timeseries batches for train, test
# using keras timeseries_dataset_from_array

sequence_length = 100 # 100ms windows
sampling_rate = 1 # sampling rate within a sequence
sequence_stride = 50 # how much to shift window by
delay = sequence_length # target is N+delay

train_dataset = {}
test_dataset = {}

for key, array in train_data_scaled_padded.items():
    
    train_dataset[key] = keras.utils.timeseries_dataset_from_array(
        data=array['raw_data'],
        targets= array['affect'],
        sequence_length=sequence_length,
        sequence_stride=sequence_stride,
        sampling_rate=sampling_rate,
        batch_size=2600, # number of sequences per batch,
        shuffle=False,
        )
    
    
    
for key, array in test_data_scaled_padded.items():
    
    test_dataset[key] = keras.utils.timeseries_dataset_from_array(
        data=array['raw_data'],
        targets=None, # set to none to generate input sequences only
        sequence_length=sequence_length,
        sequence_stride=sequence_stride,
        sampling_rate=sampling_rate,
        batch_size=610, # from 10000-100 to 40000, every 50
        shuffle=False,
        )


2023-05-01 12:47:07.018125: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 30924 MB memory:  -> device: 0, name: Tesla V100-PCIE-32GB, pci bus id: 0000:9b:00.0, compute capability: 7.0


In [ ]:
print(sorted(train_dataset.keys()))

['fold_0', 'fold_1', 'fold_2', 'fold_3', 'fold_4']


In [ ]:
#========================#
# MODEL TRAINING: LSTM   #
#========================#

# for reproducibility
tf.random.set_seed(1)

# train one model per fold
for key in sorted(train_dataset.keys()):
    print(f"Training fold: {key}")
    
    # build model
    inputs = keras.Input(shape=(100, 8))
    x = layers.Masking(mask_value=0., input_shape=(100, 8))(inputs)
    x = layers.LSTM(128, recurrent_dropout=0.25)(x)
    x = layers.BatchNormalization()(x)
    outputs = layers.Dense(2, activation="linear")(x)
    model = keras.Model(inputs, outputs)
    
    model_name = f"full_model_{key}.h5"
    print(f"Training model: {model_name}")
    
    # compile
    model.compile(optimizer="rmsprop",
                  loss="mse")
    
    # train
    model.fit(train_dataset[key],
              epochs=15,
              shuffle=False,
              verbose=1,
             )
    
    # save
    model.save(model_name)
    
    # reset for next fold
    model.reset_states()
    tf.keras.backend.clear_session()
    

Training fold: fold_0
Training model: full_model_fold_0.h5
Epoch 1/15


2023-05-01 10:28:12.831245: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_20' with dtype double and shape [24960000,2]
	 [[{{node Placeholder/_20}}]]
2023-05-01 10:28:12.831700: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype double and shape [24960000,8]
	 [[{{node Placeholder/_0}}]]


192/192 [==============================] - 43s 212ms/step - loss: 8.6465
Epoch 2/15
192/192 [==============================] - 40s 209ms/step - loss: 2.6893
Epoch 3/15
192/192 [==============================] - 40s 207ms/step - loss: 2.5908
Epoch 4/15
192/192 [==============================] - 41s 212ms/step - loss: 2.5639
Epoch 5/15
192/192 [==============================] - 40s 208ms/step - loss: 2.5501
Epoch 6/15
192/192 [==============================] - 40s 210ms/step - loss: 2.5408
Epoch 7/15
192/192 [==============================] - 40s 209ms/step - loss: 2.5460
Epoch 8/15
192/192 [==============================] - 40s 208ms/step - loss: 2.5286
Epoch 9/15
192/192 [==============================] - 40s 209ms/step - loss: 2.5221
Epoch 10/15
192/192 [==============================] - 40s 211ms/step - loss: 2.5183
Epoch 11/15
192/192 [==============================] - 40s 209ms/step - loss: 2.5204
Epoch 12/15
192/192 [==============================] - 40s 209ms/step - loss: 2.5136


2023-05-01 10:38:20.287763: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_20' with dtype double and shape [24960000,2]
	 [[{{node Placeholder/_20}}]]
2023-05-01 10:38:20.288309: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_8' with dtype int32 and shape [499199]
	 [[{{node Placeholder/_8}}]]


192/192 [==============================] - 43s 208ms/step - loss: 8.3600
Epoch 2/15
192/192 [==============================] - 40s 209ms/step - loss: 2.6637
Epoch 3/15
192/192 [==============================] - 40s 208ms/step - loss: 2.5863
Epoch 4/15
192/192 [==============================] - 41s 211ms/step - loss: 2.5643
Epoch 5/15
192/192 [==============================] - 40s 209ms/step - loss: 2.5467
Epoch 6/15
192/192 [==============================] - 40s 210ms/step - loss: 2.5343
Epoch 7/15
192/192 [==============================] - 40s 209ms/step - loss: 2.5386
Epoch 8/15
192/192 [==============================] - 40s 209ms/step - loss: 2.5335
Epoch 9/15
192/192 [==============================] - 40s 206ms/step - loss: 2.5306
Epoch 10/15
192/192 [==============================] - 40s 208ms/step - loss: 2.5339
Epoch 11/15
192/192 [==============================] - 40s 209ms/step - loss: 2.5263
Epoch 12/15
192/192 [==============================] - 40s 208ms/step - loss: 2.5214


2023-05-01 10:48:27.079288: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_28' with dtype int32 and shape [499199]
	 [[{{node Placeholder/_28}}]]
2023-05-01 10:48:27.079759: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_14' with dtype int32
	 [[{{node Placeholder/_14}}]]


192/192 [==============================] - 42s 208ms/step - loss: 8.2146
Epoch 2/15
192/192 [==============================] - 40s 208ms/step - loss: 2.6793
Epoch 3/15
192/192 [==============================] - 40s 207ms/step - loss: 2.5965
Epoch 4/15
192/192 [==============================] - 40s 208ms/step - loss: 2.5750
Epoch 5/15
192/192 [==============================] - 40s 209ms/step - loss: 2.5492
Epoch 6/15
192/192 [==============================] - 40s 208ms/step - loss: 2.5348
Epoch 7/15
192/192 [==============================] - 40s 208ms/step - loss: 2.5324
Epoch 8/15
192/192 [==============================] - 40s 207ms/step - loss: 2.5202
Epoch 9/15
192/192 [==============================] - 40s 209ms/step - loss: 2.5289
Epoch 10/15
192/192 [==============================] - 40s 208ms/step - loss: 2.5142
Epoch 11/15
192/192 [==============================] - 40s 207ms/step - loss: 2.5174
Epoch 12/15
192/192 [==============================] - 40s 207ms/step - loss: 2.5181


2023-05-01 10:58:30.392926: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_14' with dtype int32
	 [[{{node Placeholder/_14}}]]
2023-05-01 10:58:30.393394: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_8' with dtype int32 and shape [499199]
	 [[{{node Placeholder/_8}}]]


192/192 [==============================] - 42s 208ms/step - loss: 8.6424
Epoch 2/15
192/192 [==============================] - 40s 207ms/step - loss: 2.6954
Epoch 3/15
192/192 [==============================] - 40s 206ms/step - loss: 2.5917
Epoch 4/15
192/192 [==============================] - 40s 208ms/step - loss: 2.5682
Epoch 5/15
192/192 [==============================] - 40s 208ms/step - loss: 2.5493
Epoch 6/15
192/192 [==============================] - 40s 208ms/step - loss: 2.5374
Epoch 7/15
192/192 [==============================] - 40s 208ms/step - loss: 2.5377
Epoch 8/15
192/192 [==============================] - 40s 208ms/step - loss: 2.5256
Epoch 9/15
192/192 [==============================] - 40s 208ms/step - loss: 2.5212
Epoch 10/15
192/192 [==============================] - 40s 209ms/step - loss: 2.5156
Epoch 11/15
192/192 [==============================] - 39s 205ms/step - loss: 2.5165
Epoch 12/15
192/192 [==============================] - 40s 209ms/step - loss: 2.5103


In [ ]:
# # reset
# model.reset_states()
# tf.keras.backend.clear_session()

In [ ]:
#========================#
# MODEL PREDICTIONS      #
#========================#

# define a dictionary to store predictions
all_preds = {}

# loop through each model
for i in range(5):
    # load the model
    model = keras.models.load_model(f'full_model_fold_{i}.h5')
    # get predictions for the test data using the loaded model
    preds = model.predict(test_dataset[f'fold_{i}'])
    # add the predictions to the dictionary with the key 'fold_i'
    all_preds[f'fold_{i}'] = preds
    



2023-05-01 12:48:00.516774: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_13' with dtype int32
	 [[{{node Placeholder/_13}}]]
2023-05-01 12:48:00.517159: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_14' with dtype int32
	 [[{{node Placeholder/_14}}]]


48/48 [==============================] - 2s 29ms/step


2023-05-01 12:48:03.077954: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype double and shape [1444800,8]
	 [[{{node Placeholder/_0}}]]
2023-05-01 12:48:03.078347: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype double and shape [1444800,8]
	 [[{{node Placeholder/_0}}]]


48/48 [==============================] - 2s 30ms/step


2023-05-01 12:48:04.993863: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_13' with dtype int32
	 [[{{node Placeholder/_13}}]]
2023-05-01 12:48:04.994255: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype double and shape [1444800,8]
	 [[{{node Placeholder/_0}}]]


48/48 [==============================] - 2s 28ms/step


2023-05-01 12:48:06.849723: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_8' with dtype int32 and shape [28895]
	 [[{{node Placeholder/_8}}]]
2023-05-01 12:48:06.850168: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype double and shape [1444800,8]
	 [[{{node Placeholder/_0}}]]


48/48 [==============================] - 2s 29ms/step


2023-05-01 12:48:08.760272: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_8' with dtype int32 and shape [28895]
	 [[{{node Placeholder/_8}}]]
2023-05-01 12:48:08.760656: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype double and shape [1444800,8]
	 [[{{node Placeholder/_0}}]]


48/48 [==============================] - 2s 28ms/step


In [ ]:
#========================#
# SAVING PREDICTIONS     #
#========================#

# generate and save predictions as csv files

num_iterations = 48 # 48 test files
num_preds = 601 # number of predictions per file

# loop through each fold
for key, preds in sorted(all_preds.items()):
    print(key)
    print(len(preds))
    
    # save predictions
    valence = all_preds[key][:, 0:1]
    arousal = all_preds[key][:, 1:2]
    print(len(valence))
    
    # set directory file path
    directory = f"results/scenario_2/{key}/test/annotations/"
    
    # create directory if not exist
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    # loop through each test file and save
    for i in range(num_iterations):
        
        print(i)
        # get file name and set file path
        f = test_data[key][test_data[key]['time'] == 10000]['subject'].iloc[i]
        file_name = f"{f}"
        file_path = os.path.join(directory, file_name)
        
        # create time column
        df = pd.DataFrame({'time': np.arange(10000, 40000+1, 50)})
        
        start_index = (num_preds)*i
        end_index = (num_preds)*(i+1)
        
        # add in predictions
        df['valence'] = valence[start_index:end_index]
        df['arousal'] = arousal[start_index:end_index]
        
        display(df)
        
        # save df as csv
        df.to_csv(file_path, index=False)
        print(f"Saved: {file_path}")
    


    

fold_0
28895
28895
0


,time,valence,arousal
0,10000,1.534052,5.516122
1,10050,1.597782,5.507380
2,10100,1.520825,5.373878
3,10150,1.609758,5.363724
4,10200,1.632263,5.415440
...,...,...,...
596,39800,1.443349,5.689013
597,39850,1.455302,5.622540
598,39900,1.515232,5.652035
599,39950,1.464222,5.543329


Saved: results/scenario_2/fold_0/test/annotations/sub_12_vid_0.csv
1


,time,valence,arousal
0,10000,1.124032,5.641898
1,10050,1.170930,5.616934
2,10100,1.204584,5.638043
3,10150,1.203095,5.628001
4,10200,1.170762,5.626008
...,...,...,...
596,39800,1.266243,5.536883
597,39850,1.280156,5.524142
598,39900,1.282561,5.503981
599,39950,1.299027,5.495501


Saved: results/scenario_2/fold_0/test/annotations/sub_12_vid_10.csv
2


...

Saved: results/scenario_2/fold_4/test/annotations/sub_7_vid_2.csv
46


,time,valence,arousal
0,10000,3.642528,5.480839
1,10050,3.626193,5.489397
2,10100,3.654354,5.539051
3,10150,3.620232,5.495504
4,10200,3.611010,5.500579
...,...,...,...
596,39800,4.207582,5.811360
597,39850,4.630252,5.759084
598,39900,4.354972,5.847953
599,39950,4.856833,6.045923


Saved: results/scenario_2/fold_4/test/annotations/sub_7_vid_20.csv
47


,time,valence,arousal
0,10000,4.434979,5.851486
1,10050,4.293365,5.922171
2,10100,5.347320,5.714192
3,10150,5.347727,5.816393
4,10200,5.388319,5.630638
...,...,...,...
596,39800,3.670396,5.511972
597,39850,3.660645,5.534413
598,39900,3.652809,5.491939
599,39950,3.713511,5.496101


Saved: results/scenario_2/fold_4/test/annotations/sub_7_vid_9.csv
